# Data Wrangling the Toronto OpenStreetMap dataset with MongoDB

Anton Rubisov

URL to Toronto dataset: https://mapzen.com/data/metro-extracts/metro/toronto_canada/  
URL to Financial District dataset: https://www.openstreetmap.org/relation/5446604

The aim of this project is to audit (for validity, accuracy, completeness, consistency, and uniformity), clean, and explore the Toronto OpenStreetMap dataset. The full XML dump for this region is 1.2GB, and so to faciliate the auditing, I began by investigating a reduced area - the Toronto Financial District, where I work - that is only 15MB. Even here, I initially explored only 1/10th of the file in order to get a sense of where cleaning might be required.

## Stage 1: Data Cleaning

Evaluating Data Quality:  
Validity - data conforms to a schema (or other constraints)  
Accuracy - data conforms to gold standard  
Completeness - do we have all records we should have  
Consistency - consistency among fields that overlap  
Uniformity - same units used

In [1]:
import pprint
from create_reduced_dataset import *
from audit_data_quality import *

fin_dist = "financial_district.osm"
create_reduced_dataset(fin_dist, out_file="fin_dist_reduced.osm", k=10)
data_reduced = audit_key_types('fin_dist_reduced.osm')

The first thing I'll do is inspect the `node` and `way` tag keys. This'll give me a sense of what values I'm looking at, and where effort needs to be applied. It turns out there are 247 unique tag keys in just this reduced dataset, some of which have tens of entries, and a full reproduction of the `data_reduced` variable here is not appropriate - however, even a high level view of the address counts is informative:

In [2]:
display_address_counts(data_reduced)

{'addr:city': 253,
 'addr:country': 191,
 'addr:floor': 1,
 'addr:housename': 5,
 'addr:housenumber': 400,
 'addr:inclusion': 1,
 'addr:interpolation': 98,
 'addr:postcode': 31,
 'addr:province': 156,
 'addr:state': 40,
 'addr:street': 406,
 'addr:unit': 2}


This dictionary is simply counting the number of times a particular tag appears in any of the nodes. The first observation is that the number of house numbers (400) is less than the number of street names (406), indicating that 6 nodes don't even have street numbers attached to them. Extending on this, full addresses are in short supply, as the number of city, country, and postal code entries dwindles off from there. Thus, addressing the question of data completeness, many addresses are clearly incomplete. 

Even here we can see that data uniformity is lacking: some addresses have a 'state' value while many more have a 'province' value. Being as we are in Canada, all addresses should have a province rather than a state.

Exploring the address data a little further for validity, both the city and province attributes need to be standardized:

In [3]:
print(data_reduced['addr:city'])
print(data_reduced['addr:province'])

{'count': 253, 'vals': {'Toronto', 'City of Toronto'}}
{'count': 156, 'vals': {'Ontario', 'ON'}}


Luckily, postal codes, where available, are in good order and format. 

Another small address issue, also with data validity, is found in street names. The street that runs along the harbor front is called Queens Quay, and appears in the dictionary of street names under both that name and Queen's Quay. This is hardly surprising, as Torontonians seldom appreciate the distinction, and it even appears under both guises in Wikipedia: [Queens Quay](https://en.wikipedia.org/wiki/Queens_Quay_(Toronto&#41;) and [Queen's Quay Terminal](https://en.wikipedia.org/wiki/Queen%27s_Quay_Terminal). This'll likely be an issue with the even more misunderstood [Princes' Gates](https://en.wikipedia.org/wiki/Exhibition_Place#Princes.27_Gates).

Two other areas that I will focus on for cleanup are `phone` and `opening_hours`. Here the data does not conform to a single schema for format, and this will need to be standardized in the cleanup phase:

In [4]:
pprint.pprint(data_reduced['phone'])
pprint.pprint(data_reduced['opening_hours'])

{'count': 25,
 'vals': {'(416) 367-0685',
          '(647) 896-1774',
          '+1 (416) 362-1739',
          '+1 (416) 365-0481',
          '+1 (647) 352 8802',
          '+1 416 642 8321',
          '+1 416 861-1211',
          '+1 416 864 7791',
          '+1 416 916 8811',
          '+1 416-977-6000 Ext. 2208',
          '+1-416-597-0366',
          '+1-416-865-9700',
          '+1-416-913-8880',
          '+1-416-941-8920',
          '416 862 7575',
          '416-306-0335',
          '416-348-8887',
          '416-367-0645',
          '416-546-2200',
          '416-591-0404',
          '416-593-2560',
          '416-599-3334',
          '416-862-8899',
          '416.977.3222',
          '4163638330'}}
{'count': 15,
 'vals': {'24/7',
          'Minday-Friday : 08:00-19:00',
          'Mo-Fr 06:30-18:00; Sa-Su 10:00-16:00',
          'Mo-Fr 07:00-23:00',
          'Mo-Fr 08:00-18:00; Tu,Th 08:00-19:00; Sa 09:00-13:00',
          'Mo-Fr 6:30-19:00; Sa-Su 8:00-18:00',
          'Mo

To address the phone number format, we look to the Canadian national convention for phone numbers: http://www.bt-tb.tpsgc-pwgsc.gc.ca/btb.php?lang=eng&cont=044 As the OpenStreetMap project extends to an international audience, we'll follow the guideline for international formatting: "The Translation Bureau recommends writing a Canadian telephone number in the following international format, without hyphens: + 1 819 555 5555" Thus our phones get reformatted as:

In [5]:
from format_phones import *

for phone in data_reduced['phone']['vals']:
    print("{:>25}  ->  {}".format(phone,parse_phone_number(phone)))

             416-599-3334  ->  +1 416 599 3334
             416-591-0404  ->  +1 416 591 0404
             416-593-2560  ->  +1 416 593 2560
           (647) 896-1774  ->  +1 647 896 1774
             416 862 7575  ->  +1 416 862 7575
          +1 416 916 8811  ->  +1 416 916 8811
        +1 (647) 352 8802  ->  +1 647 352 8802
        +1 (416) 362-1739  ->  +1 416 362 1739
             416-306-0335  ->  +1 416 306 0335
          +1-416-597-0366  ->  +1 416 597 0366
+1 416-977-6000 Ext. 2208  ->  +1 416 977 6000 Ext. 2208
          +1-416-913-8880  ->  +1 416 913 8880
          +1-416-941-8920  ->  +1 416 941 8920
          +1 416 642 8321  ->  +1 416 642 8321
             416-546-2200  ->  +1 416 546 2200
             416.977.3222  ->  +1 416 977 3222
             416-862-8899  ->  +1 416 862 8899
             416-367-0645  ->  +1 416 367 0645
        +1 (416) 365-0481  ->  +1 416 365 0481
             416-348-8887  ->  +1 416 348 8887
           (416) 367-0685  ->  +1 416 367 0685
   

Turning to opening hours, this format is a lot more difficult to massage into a single schema. According to http://wiki.openstreetmap.org/wiki/Key:opening_hours, we want days of the week identified by a two-character abbreviation; day ranges identified by a hyphen and multiple dates separated by a comma (e.g. Mo-Fr and Tu,Th); times by the 24h clock with a leading zero if before noon; multiple time ranges per entry separated by a comma (e.g. 00:00-01:00,11:00-24:00); multiple entries separated by a semi-colon (e.g. Mo-Fr 06:30-18:00; Sa-Su 10:00-16:00); closures identified by absence of entries rather than an 'off' entry (e.g. Sa-Su off); and no overlap in the dates in the entries (e.g. Mo-Fr 08:00-18:00; Tu,Th 08:00-19:00 overlaps Tu and Th because they're also in the Mo-Fr range). And those are just the simple values - the documentation gets vastly more complex in treating odd cases, which for this project we'll just leave unchanged. Thus, our data becomes:

In [6]:
from format_hours import *

for hours in data_reduced['opening_hours']['vals']:
    print("    {}\n->  {}\n".format(hours,parse_opening_hours(hours)))

    Mo-Fr 8:00-17:00
->  Mo-Fr 08:00-17:00

    We-Mo 07:00-20:00, Tu 07:00-19:00
->  Mo,We-Su 07:00-20:00; Tu 07:00-19:00

    Minday-Friday : 08:00-19:00
->  Mo-Fr 08:00-19:00

    Mo-Sa 10:00-18:00; Su off
->  Mo-Sa 10:00-18:00

    Mo-Fr 06:30-18:00; Sa-Su 10:00-16:00
->  Mo-Fr 06:30-18:00; Sa-Su 10:00-16:00

    Mo-Fr 07:00-23:00
->  Mo-Fr 07:00-23:00

    Mo-Su 11:00-22:00
->  Mo-Su 11:00-22:00

    Mo-Fr 7:45-17:00; Sa-Su off
->  Mo-Fr 07:45-17:00

    Mo-Fr 08:00-18:00; Tu,Th 08:00-19:00; Sa 09:00-13:00
->  Mo,We,Fr 08:00-18:00; Tu,Th 08:00-19:00; Sa 09:00-13:00

    Mo-Th 00:00-01:00,11:00-24:00; Fr-Su 00:00-02:00,11:00-24:00
->  Mo-Th 00:00-01:00,11:00-24:00; Fr-Su 00:00-02:00,11:00-24:00

    Mo-Fr 9:00-19:00; Sa-Su off
->  Mo-Fr 09:00-19:00

    Mo-Fr 6:30-19:00; Sa-Su 8:00-18:00
->  Mo-Fr 06:30-19:00; Sa-Su 08:00-18:00

    24/7
->  Mo-Su 00:00-24:00



Thanks to Regex, the code in fact does a rather good job of getting the opening hours into the format that we want, including capturing hours on public holidays.

The majority of the strings have critical information missing (e.g. open Mo-Su but at what hours? Open 11:00-23:00 but on what days?). The last few strings require parsing the 12-hour clock times into 24-hour times, as well as dealing with the additional hyphen between the day names and times, and I leave this to a future iteration of this project.

In summary, our data cleanup at this stage will require the following:
1. Parsing and expanding abbreviations in street names (incl. mapping Queen's Quay to Queens Quay)
2. Filling in missing city, province, and country values
3. Updating existing city, province, and country values to conform to a single schema
4. Updating existing phone numbers to conform to a single schema
5. Updating business opening hours to conform to a single schema

Once cleaned, the Toronto data was converted into a JSON format and pushed into a MongoDB database using the command:  

<code>mongoimport --db udacity --collection toronto --file toronto_canada.osm.json</code>

(A note on using ElementTree.IterParse: due to the size of the OSM file, the cleaning script perpetually froze my computer when attempting to work with the full dataset - after a couple reboots, I determined this was from the script utilizing all my RAM. The solution was:  
<pre>
for _, element in ET.iterparse(file_in, events=("start",)):
    # if element.tag == 'node':
        ...
        
    element.clear() 
</pre>

Clearing the element from memory eliminated RAM utilization entirely.)

## Stage 2: Overview of Data in MongoDB

This section contains basic statistics about the dataset and the MongoDB queries used to gather them.

<pre>File sizes
                                                
toronto_canada.osm ......... 1182 MB
toronto_canada.osm.json .... 1299 MB</pre>

The following presents some of the basic statistics on the dataset obtained by running the mongo shell in bash:

#### Number of records
<code> \> db.toronto.find().count()
5647295
</code>

#### Number of nodes
<code> \> db.toronto.find({"type":"node"}).count()
4922735
</code>

#### Number of ways
<code> \> db.toronto.find({"type":"way"}).count()
720747
</code>
                                                
#### Number of unique contributors
<code> \> db.toronto.distinct("created.user").length
2179
</code>

#### Being as we are in Canada -- number of Tim Hortons
<code> \> db.toronto.find({"name":"Tim Hortons"}).count()
611
</code>

We'll explore some of the data a little further, and for the sake of documentation, access some of the figures using queries through PyMongo:

In [7]:
from mongo_interface import *
db = get_db('udacity')

# Of the contents of this dataset, which extends beyond the municipal boundaries
# of the City of Toronto, what proportion fall into which township?
pipeline = [{'$match': {'address': {'$exists': 1}}},
            {'$group': {'_id': '$address.city',
                        'count': {'$sum': 1} } },
            {'$sort': { 'count': -1 } },
            {'$limit': 10}]
pprint(aggregate(db, pipeline))

[{'_id': None, 'count': 286160},
 {'_id': 'Toronto', 'count': 119847},
 {'_id': 'Hamilton', 'count': 40332},
 {'_id': 'Mississauga', 'count': 36935},
 {'_id': 'Brampton', 'count': 28856},
 {'_id': 'Vaughan', 'count': 17332},
 {'_id': 'Town of Markham', 'count': 16732},
 {'_id': 'Burlington', 'count': 15944},
 {'_id': 'Oakville', 'count': 15637},
 {'_id': 'Oshawa', 'count': 13930}]


In [8]:
# Limiting ourselves to just Toronto proper, what restaurants are open 24/7?
pipeline = [{'$match': {'address.city': 'Toronto',
                        'opening_hours': 'Mo-Su 00:00-24:00',
                        'amenity': {'$in': ['fast_food','cafe','restaurant'] }}},
            {'$group': {'_id': '$name',
                        'count': {'$sum': 1} } },
            {'$sort': { 'count': -1 } }]
pprint(aggregate(db, pipeline))

[{'_id': "McDonald's", 'count': 9},
 {'_id': 'Tim Hortons', 'count': 8},
 {'_id': 'The Bagel House', 'count': 2},
 {'_id': 'A&W', 'count': 2},
 {'_id': '7 West', 'count': 1},
 {'_id': "Fran's Restaurant", 'count': 1}]


In [9]:
# What is the size of the transport networks in the dataset?
pipeline = [{'$match': {'$or': [ {'highway': 'bus_stop'}, {'railway': 'tram_stop'}] } },
            {'$group': {'_id': '$operator',
                        'count': {'$sum': 1} } },
            {'$sort': {'count': -1}},
            {'$limit': 20}]
pprint(aggregate(db, pipeline))

[{'_id': 'Toronto Transit Commission', 'count': 5343},
 {'_id': 'York Region Transit', 'count': 1562},
 {'_id': 'MiWay', 'count': 1531},
 {'_id': 'Durham Region Transit', 'count': 688},
 {'_id': None, 'count': 450},
 {'_id': 'Hamilton Street Railway', 'count': 355},
 {'_id': 'Brampton Transit', 'count': 323},
 {'_id': 'Oakville Transit', 'count': 259},
 {'_id': 'GO Transit', 'count': 229},
 {'_id': 'Burlington Transit', 'count': 134},
 {'_id': 'Milton Transit', 'count': 126},
 {'_id': 'VIVA', 'count': 73},
 {'_id': 'York Region Transit, GO Transit', 'count': 64},
 {'_id': 'York Region Transit (contracted)', 'count': 48},
 {'_id': 'Durham Region Transit; GO Transit', 'count': 40},
 {'_id': 'Toronto Transit Commission, York Region Transit', 'count': 40},
 {'_id': 'Zum (Brampton Transit)', 'count': 27},
 {'_id': 'York Region Transit, VIVA', 'count': 23},
 {'_id': 'Toronto Transit Commision', 'count': 22},
 {'_id': 'Durham Region Transit, GO Transit', 'count': 19}]


In [10]:
# Yonge Street is the longest street in the world - does it also contain the most nodes?
pipeline = [{'$match': {'type': 'node',
                        'address.street': {'$exists': 1} } },
            {'$group': {'_id': '$address.street',
                        'count': {'$sum': 1} } },
            {'$sort': {'count': -1}},
            {'$limit': 3}]
pprint(aggregate(db, pipeline))

[{'_id': 'Yonge Street', 'count': 2267},
 {'_id': 'Bathurst Street', 'count': 1089},
 {'_id': 'Dundas Street West', 'count': 1035}]


## Stage 3: Ideas for Improvement

The central tenet of this project, that human input leads to errors in data input which impedes data analysis, has made itself painfully clear in this project. Things like misplaced apostrophes lurk throughout, and typos like 'Commision' instead of 'Commission' lead to, in the output immediately above, 22 bus stops being miscategorized. Perhaps the simplest improvement that can be made is a parsing every key:value pair for typos via a dictionary/autosuggest library to probabilistically correct typos - such as, e.g., the NLTK library.

The opening_hours field, investigated in detail in this project, was a very interesting and complex source of issues. Users submitted information in just about every format imaginable, and as the proper schema proposed by OSM has many rules and many special cases, parsing the values programatically is a challenge. There appears to be an open source Python script on Git, found at https://github.com/opening-hours/opening_hours.js/blob/master/regex_search.py, which attempts more sophisticated parsing that I've developed for this project, and it may be worthwhile to push my work here as a contribution toward that project.

Perhaps the greatest challenge in working with this dataset is the profusion of missing data. As an example, for nodes containing addresses, the most common 'city' value is actually a missing value! In theory, this value should be derivable from the latitude and longitude of a given node, so long as there is a 'gold standard' value for municipal boundaries in these terms - and, as boundaries are seldom linear, this could take substantial manual labour to encode. Once defined, though, any node's address.city value can be easily determined. 

## Resources Used:
1. https://docs.mongodb.com
2. https://docs.python.org/3/library/xml.etree.elementtree.html
3. https://regex101.com/
4. http://stackoverflow.com/questions/7697710/python-running-out-of-memory-parsing-xml-using-celementtree-iterparse
5. https://www.quora.com/What-is-the-best-way-to-parse-and-standardize-phone-numbers-in-Python/answer/Stuart-Woodward?srid=1FEf
6. http://stackoverflow.com/questions/5906984/combine-days-where-opening-hours-are-similar